In [ ]:
import math
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import itertools

from ema_workbench import save_results, load_results
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import clusterer, plotting, Density

#%matplotlib inlin
clr_palette = ([sns.color_palette("YlGn", 15)[10],sns.cubehelix_palette(8)[6]])

pydice_folder = "C:/Users/ivart/OneDrive/Bureaublad/Afstuderen/WRR/PyRICE Ivar Tjallingii/PyRICE2020/6_Uncertainty Analysis/server/model_server"
print(pydice_folder)
sys.path.append(pydice_folder)

from ema_workbench import (perform_experiments, Model, Policy, Scenario, ReplicatorModel, RealParameter, IntegerParameter, ScalarOutcome, ArrayOutcome, 
                           Constant, ema_logging, SequentialEvaluator, MultiprocessingEvaluator, IpyparallelEvaluator)

from PyRICE_V8_long_term_uncertainty import PyRICE

from matplotlib import pyplot

In [ ]:
from model_outcomes_uncertainty_v2 import get_all_model_outcomes_uncertainty_search

In [ ]:
timepoints_to_save = np.arange(2005,2305+20,20)
objectives_list_name = ['Intertemporal utility GINI','Intertemporal impact GINI','Total Aggregated Utility','Regions below treshold']

objectives_list_timeseries_name = ['Damages ','Utility ',
            'Lowest income per capita ','Highest climate impact per capita ',
            'Distance to treshold ','Population under treshold ',
            'Intratemporal utility GINI ','Intratemporal impact GINI ',
            'Atmospheric Temperature ', 'Industrial Emission ', 'Total Output ']

supplementary_list_timeseries_name = ['CPC ','Population ']
supplementary_list_quintile_name = ['CPC pre damage ','CPC post damage ']

outcomes_names_time_series = []
for name in objectives_list_timeseries_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcomes_names_time_series.append(name_year)

outcomes_name_single = []
outcomes_name_multiple= []
for name in objectives_list_name:
    if name == "Regions below treshold":
        outcomes_name_multiple.append(name)
    else:
        outcomes_name_single.append(name)

outcome_name_supplementary =[]
for name in supplementary_list_timeseries_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcome_name_supplementary.append(name_year)
        
outcome_name_supplementary_quintile=[]
for name in supplementary_list_quintile_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcome_name_supplementary_quintile.append(name_year)
        

In [ ]:
RICE_POP_gr = pd.read_excel("RICE_2010_base_000.xlsm", sheet_name="Pop_gr")

regions_list = ["US", "OECD-Europe","Japan","Russia","Non-Russia Eurasia","China","India","Middle East","Africa",
    "Latin America","OHI","Other non-OECD Asia"]

"""
####################### Population PARAMETERS and set up dataframe format #######################
"""

#get population growth rates for each region
a=[]
for i in range(31):  
    if i == 0:
        a.append("region")
    k = 2005 + 10 * i
    k = str(k)
    a.append(k)    

region_pop_gr = RICE_POP_gr.iloc[10:22,3:35]
region_pop_gr.columns =  a
region_pop_gr = region_pop_gr.set_index('region') 


In [ ]:
all_policies = pd.read_csv("example_policys_principles.csv",index_col=0)

principles_list =["utilitarian","prioritarian","egalitarian","sufficitarian","nordhaus"]

total_policy_list = []

for principle in principles_list:
    policies = all_policies[all_policies['principle']==principle]
    policies = policies.dropna(axis='columns')
    policies = policies.iloc[:,:-1]
    policy_list_principle = []

    #get list of policies as input for uncertainty sampling
    for i in range(0,len(policies)):
        policy_dict = policies.iloc[i].to_dict()
        policy_list_principle.append(Policy(policies.index[i], **policy_dict)) 
    total_policy_list.append(policy_list_principle)
total_policy_list[4]

In [ ]:
from load_result import load_result

In [ ]:
result_nordhaus = load_result('results_uncertainty_analsysis_long_term_local_test_nordhaus_runs_25000.tar.gz')
experiments_nordF, outcomes_nord = result_nordhaus

In [ ]:
result_util = load_result('results_uncertainty_analsysis_long_term_utilitarian_runs_25000.tar.gz')
experiments_util, outcomes_util= result_util

In [ ]:
result_ega2 = load_result('results_uncertainty_analsysis_long_term_policies_second_halfegalitarian_runs_25000.tar.gz')
experiments_ega2, outcomes_ega2 = result_ega2

In [ ]:
result_ega = load_result('results_uncertainty_analsysis_long_term_policiesegalitarian_runs_25000.tar.gz')
experiments_ega, outcomes_ega = result_ega

In [ ]:
results_suf2 = load_result('results_uncertainty_analsysis_long_term_policies_second_halfsufficitarian_runs_25000.tar.gz')
experiments_suf2, outcomes_suf2 = results_suf2

In [ ]:
results_suf = load_result('results_uncertainty_analsysis_long_term_policiessufficitarian_runs_25000.tar.gz')
experiments_suf, outcomes_suf = results_suf

In [ ]:
results_prio2 = load_result('results_uncertainty_analsysis_long_term_policies_second_halfprioritarian_runs_25000.tar.gz')
experiments_prio2, outcomes_prio2 = results_prio2

In [ ]:
results_prio = load_result('results_uncertainty_analsysis_long_term_policiesprioritarian_runs_25000.tar.gz')
experiments_prio, outcomes_prio = results_prio

### get per region outputs and save in CSV

In [ ]:
experiments_total= pd.DataFrame()

#experiments_nord, outcomes_nord = results_nordhaus
experiments_ega2["principle"] = "Egalitarian"
experiments_suf2["principle"] = "Sufficitarian"
experiments_prio2["principle"] = "Prioritarian"

experiments_util["principle"] = "Utilitarian"
experiments_ega["principle"] = "Egalitarian"
experiments_prio["principle"] = "Prioritarian"
experiments_suf["principle"] = "Sufficitarian"
experiments_nordF["principle"] = "Nordhaus policy"

outcomes_names_main = outcomes_names_time_series+outcomes_name_single

for keys in outcomes_names_main:
    experiments_ega[keys] = outcomes_ega[keys]
    experiments_util[keys] = outcomes_util[keys]
    experiments_suf[keys] = outcomes_suf[keys]
    experiments_prio[keys] = outcomes_prio[keys]
    experiments_nordF[keys] = outcomes_nord[keys]
    experiments_suf2[keys] = outcomes_suf2[keys]
    experiments_ega2[keys] = outcomes_ega2[keys]
    experiments_prio2[keys] = outcomes_prio2[keys]


experiments_total = pd.concat([experiments_prio2,experiments_suf2,experiments_ega2,experiments_ega,experiments_suf,experiments_prio,experiments_util,
                               experiments_nordF],axis=0)

In [ ]:
experiments_total.to_csv("long_term_uncertainty_experiments_total_complete.csv")

### get per region outputs and save in CSV

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary[16:32]

tuples = ()
index = pd.MultiIndex.from_tuples(tuples,names=['principle', 'year'])
POP_total = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','sufficitarian','utilitarian','prioritarian']
outcomes = [outcomes_ega,outcomes_suf,outcomes_util,outcomes_prio]

index_year = 0
principle_index =0 

for outcome_principle in outcomes:
    for year in pop_keys:
        length = (len(outcome_principle[year]))
        if principles[principle_index] =="utilitarian":
            tuples = [(principles[principle_index],2005+index_year),]*(125*10**3)
        else:
            tuples = [(principles[principle_index],2005+index_year),]*10**5
        index_1 = pd.MultiIndex.from_tuples(tuples, names=['principle', 'year'])
        year_df = pd.DataFrame(data =outcome_principle[year], columns= region_pop_gr.index,index=index_1)

        index_year = index_year +20
        POP_total = pd.concat([POP_total,year_df],axis=0)
    principle_index = principle_index + 1

In [ ]:
POP_total.to_csv("long_term_POP_per_time_period.csv")

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary[0:16]

tuples = ()
index = pd.MultiIndex.from_tuples(tuples,names=['principle', 'year'])
CPC_total = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','sufficitarian','utilitarian','prioritarian']
outcomes = [outcomes_ega,outcomes_suf,outcomes_util,outcomes_prio]

index_year = 0
principle_index =0 

for outcome_principle in outcomes:
    for year in pop_keys:
        length = (len(outcome_principle[year]))
        
        if principles[principle_index] =="utilitarian":
            tuples = [(principles[principle_index],2005+index_year),]*(125*10**3)
        else:
            tuples = [(principles[principle_index],2005+index_year),]*10**5
        index_1 = pd.MultiIndex.from_tuples(tuples, names=['principle', 'year'])
        year_df = pd.DataFrame(data =outcome_principle[year], columns= region_pop_gr.index,index=index_1)

        index_year = index_year +20
        CPC_total = pd.concat([CPC_total,year_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
CPC_total.to_csv("long_term_POP_per_time_period.csv")

### get per quintile outputs and save in CSV

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary_quintile[0:16]

quintile = 0
tupple = []
for i in range(0,10):
    run = i
    if quintile ==5:
        quintile = 0
    tupple_to_append = ('egal',year,run,quintile)
    tupple.append(tupple_to_append)
    quintile = quintile + 1
    
index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])
CPC_total = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','sufficitarian','utilitarian','prioritarian']
outcomes = [outcomes_ega,outcomes_suf,outcomes_util,outcomes_prio]

index_year = 0
principle_index =0 

for principle in principles:
    outcome = outcomes[principle_index]
    for year in pop_keys:
        #set up tupple 
        quintile = 0
        tupple = []

        for i in range(0,len(outcome[year])*5):
            run = i
            
            if quintile ==5:
                quintile = 0
                
            tupple_to_append = (principle,year,run,quintile)
            tupple.append(tupple_to_append)
            quintile = quintile + 1

        #build index for dataframe to store results per saved timestep 
        index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])

        CPC_df = pd.DataFrame(data = outcome[year].reshape(-1,data.shape[-1]),columns= region_pop_gr.index,index =index)

        CPC_total = pd.concat([CPC_total,CPC_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
CPC_total.to_csv("long_term_CPC_before_damages_per_time_period.csv")

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary_quintile[16:32]

quintile = 0
tupple = []
for i in range(0,10):
    run = i
    if quintile ==5:
        quintile = 0
    tupple_to_append = ('egal',year,run,quintile)
    tupple.append(tupple_to_append)
    quintile = quintile + 1
    
index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])
CPC_total_after = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','sufficitarian','utilitarian','prioritarian']
outcomes = [outcomes_ega,outcomes_suf,outcomes_util,outcomes_prio]

index_year = 0
principle_index =0 

for principle in principles:
    outcome = outcomes[principle_index]
    for year in pop_keys:
        #set up tupple 
        quintile = 0
        tupple = []

        for i in range(0,len(outcome[year])*5):
            run = i
            
            if quintile ==5:
                quintile = 0
                
            tupple_to_append = (principle,year,run,quintile)
            tupple.append(tupple_to_append)
            quintile = quintile + 1

        #build index for dataframe to store results per saved timestep 
        index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])

        CPC_df = pd.DataFrame(data = outcome[year].reshape(-1,data.shape[-1]),columns= region_pop_gr.index,index =index)

        CPC_total_after = pd.concat([CPC_total,CPC_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
CPC_total_after.to_csv("long_term_CPC_after_damages_per_time_period.csv")